
# 03 · Reporting

Dieses Notebook erzeugt **nachvollziehbare, wissenschaftlich korrekte** Abbildungen:
- Niveau-Plots je Jahr mit **Prozentwerten an jedem Balken**
- Gruppenbalken 2021/2022/2024
- **Δ in Prozentpunkten (pp)** 2021→2024 als horizontales Ranking
- Dumbbell-Plot 2021→2024
- Auswertung **Bedarfsgüter vs. Luxusgüter** (anpassbare Einteilung)

Außerdem wird ein **Kurzbefund** als Textdatei exportiert.


In [28]:

# 03_reporting — Cell 1: Setup & Daten laden
from __future__ import annotations

from pathlib import Path
import json, numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

BASE = Path.cwd().resolve().parents[0] if Path.cwd().name.lower()=="notebooks" else Path.cwd()
OUT  = BASE / "data" / "processed"
FIG  = BASE / "reports" / "figures"
FIG.mkdir(parents=True, exist_ok=True)

CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]

WIDE_PATH = OUT / "statista_harmonisiert_2021_2022_2024.csv"
LONG_PATH = OUT / "statista_long_2021_2022_2024.csv"

stat_pivot = pd.read_csv(WIDE_PATH).set_index("Kategorie").reindex(KANON).fillna(0.0)
stat_all   = pd.read_csv(LONG_PATH)

YEARS = sorted([int(c) for c in stat_pivot.columns if str(c).isdigit()])
print("Geladene Jahre:", YEARS)
display(stat_pivot.head(7))


Geladene Jahre: [2021, 2022, 2024]


,2021,2022,2024
Kategorie,,,
Kleidung / Schuhe,57.0,78.0,106.0
"Elektronik (z. B. Smartphones, Haushaltsgeräte)",38.0,145.0,38.0
Lebensmittel / Getränke,33.0,28.0,16.0
Bücher / Medien / Software,0.0,92.0,25.0
Medikamente / Drogerieartikel,78.0,92.0,51.0
Hobby- & Freizeitartikel,83.0,71.0,57.0
Möbel / Wohnaccessoires,38.0,32.0,15.0


In [29]:
# Spaltennamen robust in ints umwandeln (z.B. "2021", " 2021 ", "2021.0" -> 2021)
def _to_int_if_year(c):
    s = str(c).strip()
    if s.endswith(".0") and s[:-2].isdigit():
        s = s[:-2]
    return int(s) if s.isdigit() else s

stat_pivot.columns = [ _to_int_if_year(c) for c in stat_pivot.columns ]
YEARS = sorted([c for c in stat_pivot.columns if isinstance(c, int)])
print("YEARS (int):", YEARS)


YEARS (int): [2021, 2022, 2024]


In [30]:
# Einheitlicher Stil + robustes Layout
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 11,
    "axes.titlesize": 12,
    "axes.labelsize": 11,
    "axes.grid": True,
    "grid.alpha": 0.2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    # Empfehlung: kein autolayout, wir nutzen constrained_layout gezielt pro Figure
    "figure.autolayout": False,
})

def percent_axis(ax, axis="y", decimals=0, limit=(0, 100), pad_pct=0):
    """
    Prozent-Achse 0–100 % mit optionaler rechter/linker Padding-Reserve (pad_pct in %-Punkten),
    damit äußere Labels nicht abgeschnitten werden.
    """
    lo, hi = limit
    hi_padded = hi + float(pad_pct)
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "y":
        ax.set_ylim(lo, hi_padded)
        ax.yaxis.set_major_formatter(fmt)
    else:
        ax.set_xlim(lo, hi_padded)
        ax.xaxis.set_major_formatter(fmt)

def label_bars(ax, decimals=1, rotate=0, inside=False):
    """
    Beschriftet Balken. inside=True platziert Werte IN den Balken (vermeidet Abschneiden).
    """
    fmt = f"%.{decimals}f %%"
    for c in ax.containers:
        if inside:
            ax.bar_label(c, fmt=fmt, padding=2, fontsize=8, rotation=rotate, label_type="center")
        else:
            ax.bar_label(c, fmt=fmt, padding=2, fontsize=8, rotation=rotate)

def label_hbars_right(ax, bars, decimals=1, dx=1.0):
    """
    Beschriftet H-Balken RECHTS außerhalb, mit kleiner Reserve dx (in %-Punkten).
    Setzt clip_on=False, damit nichts „hart“ an der Achse abgeschnitten wird.
    """
    for rect in bars:
        v = rect.get_width()
        ax.text(v + dx, rect.get_y() + rect.get_height()/2,
                f"{v:.{decimals}f} %", va="center", ha="left", fontsize=9, clip_on=False)

def footer(note: str):
    # Fußnote leicht nach unten, wird durch bbox_inches='tight' mitgenommen
    plt.figtext(0.01, -0.04, note, ha="left", va="top", fontsize=9)

def save_fig(path):
    # 'tight' + pad verhindert Abschneiden (Legende, lange Labels)
    plt.savefig(path, bbox_inches="tight", pad_inches=0.25, facecolor="white")
    plt.close()


In [31]:

# 03_reporting — Cell 3: Niveau-Plots je Jahr (mit Prozentlabels)
for yr in YEARS:
    values = stat_pivot[yr].sort_values(ascending=True)
    fig, ax = plt.subplots(figsize=(12, 6), layout="constrained")
# KEIN plt.tight_layout() zusätzlich! (beides zusammen kann stören)
    bars = ax.barh(values.index, values.values)
    for rect, v in zip(bars, values.values):
        ax.text(rect.get_width() + 1.0, rect.get_y() + rect.get_height()/2,
                f"{v:.1f} %", va="center", fontsize=9)
    ax.set_title(f"Online-Kaufanteile nach Produktkategorien – {yr}")
    ax.set_xlabel("Anteil in %")
    percent_axis(ax, axis="x", decimals=0, limit=(0, 100))
    plt.tight_layout()
    footer(f"Quelle: Statista ({yr}). Einheit: Prozent. Hinweis: Harmonisierung gemäß Mapping; Teilkategorien konsolidiert.")
    save_fig(FIG / f"03_{yr}_niveau.png")
print("Niveauabbildungen exportiert →", FIG)


C:\Users\admin\AppData\Local\Temp\ipykernel_26376\335776326.py:13: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Niveauabbildungen exportiert → D:\Q3_2025\data-analytics\project\reports\figures


In [43]:
# 03_reporting — Cell 4: Gruppenbalken (2021/2022/2024) inkl. Prozentlabels — FIX
import numpy as np
fig, ax = plt.subplots(figsize=(13, 6), constrained_layout=True)   # nur EINE Layout-Engine

# Plot
(stat_pivot[YEARS].plot(kind="bar", ax=ax, width=0.8, zorder=2))

# --- Headroom für Labels oberhalb der Balken ---
ymax = float(np.nanmax(stat_pivot[YEARS].values))
ax.set_ylim(0, max(100, ymax * 1.18))   # 18% Luft; aber min 100%, falls Achse auf 0–100 soll

# Prozentlabels (achte darauf, dass deine label_bars nicht 'clip_on' erzwungen auf True setzt)
label_bars(ax, decimals=1, rotate=0)  # Labels bleiben horizontal lesbar; kein Rotate nötig

# Achsen/Beschriftungen
ax.set_title("Online-Kaufanteile nach Kategorien – Vergleich 2021 / 2022 / 2024")
ax.set_xlabel("Kategorie")
ax.set_ylabel("Anteil in %")
percent_axis(ax, axis="y", decimals=0, limit=(0, 150))

# xtick-Labels: leicht rotieren und rechts ausrichten
plt.xticks(rotation=28, ha="right")

# Legende oberhalb, kollisionsfrei außerhalb
ax.legend(
    title="Jahr",
    ncols=len(YEARS),
    frameon=False,
    loc="lower center",
    bbox_to_anchor=(0.5, 1.02)
)

# KEIN plt.tight_layout() hier!
# Fußnote außerhalb der Achse; footer sollte intern fig.text(...) verwenden
def footer(text, fig=None, y=-0.04):
    """Fußnote außerhalb der Achsen – kollisionsfrei mit constrained_layout."""
    import matplotlib.pyplot as plt
    if fig is None:
        fig = plt.gcf()
    fig.text(0.01, y, text, ha="left", va="top", fontsize=9, color="#555")

footer("Quelle: Statista (2021–2024, ohne 2023). Einheit: Prozent. Hinweis: Harmonisierung gemäß Mapping; Teilkategorien konsolidiert.",
       fig=fig)

save_fig(FIG / "03_vergleich_2021_2022_2024.png")


In [46]:

# 03_reporting — Cell 5: Δ in Prozentpunkten (pp)
delta = pd.DataFrame(index=stat_pivot.index)
if set([2021,2022,2024]).issubset(stat_pivot.columns):
    delta["Δ 2021→2022"] = stat_pivot[2022] - stat_pivot[2021]
    delta["Δ 2022→2024"] = stat_pivot[2024] - stat_pivot[2022]
    delta["Δ 2021→2024"] = stat_pivot[2024] - stat_pivot[2021]
else:
    # Fallback, falls Jahre fehlen
    years_sorted = sorted([int(c) for c in stat_pivot.columns])
    delta[f"Δ {years_sorted[0]}→{years_sorted[-1]}"] = stat_pivot[years_sorted[-1]] - stat_pivot[years_sorted[0]]
delta = delta.sort_values(delta.columns[-1], ascending=False)

display(delta.style.format("{:.1f}"))

DELTA_PATH = OUT / "statista_delta_2021_2022_2024.csv"
delta.to_csv(DELTA_PATH, encoding="utf-8")
print("Delta-Tabelle exportiert →", DELTA_PATH)


,Δ 2021→2022,Δ 2022→2024,Δ 2021→2024
Kategorie,,,
Kleidung / Schuhe,21.0,28.0,49.0
Bücher / Medien / Software,92.0,-67.0,25.0
"Elektronik (z. B. Smartphones, Haushaltsgeräte)",107.0,-107.0,0.0
Lebensmittel / Getränke,-5.0,-12.0,-17.0
Möbel / Wohnaccessoires,-6.0,-17.0,-23.0
Hobby- & Freizeitartikel,-12.0,-14.0,-26.0
Medikamente / Drogerieartikel,14.0,-41.0,-27.0


Delta-Tabelle exportiert → D:\Q3_2025\data-analytics\project\data\processed\statista_delta_2021_2022_2024.csv


In [48]:
# 03_reporting — Cell 6: Balken (pp) mit Vorzeichen, symmetrische Skala
col = delta.columns[-1]           # z.B. "Δ 2021→2024"
order = delta.index               # Reihenfolge wie zuvor sortiert
vals = delta[col].reindex(order)  # Serie mit Deltas

fig, ax = plt.subplots(figsize=(12, 6), layout="constrained")  # kein tight_layout zusätzlich!
bars = ax.barh(order, vals.values, color=["#4d8" if v >= 0 else "#d66" for v in vals.values])

# Labels mit Vorzeichen, minimaler Versatz; clip_on=False verhindert hartes Abschneiden
for rect, v in zip(bars, vals.values):
    ax.text(
        v + (0.5 if v >= 0 else -0.5),
        rect.get_y() + rect.get_height()/2,
        f"{v:+.1f} pp",
        va="center",
        ha="left" if v >= 0 else "right",
        fontsize=9,
        clip_on=False,
    )

ax.set_title(f"Veränderung der Online-Kaufanteile ({col.split('Δ')[-1].strip()})")
ax.set_xlabel("Δ in Prozentpunkten (pp)")
left  = min(-10, float(vals.min()) - 23)
right = max( 10, float(vals.max()) + 1)
ax.set_xlim(left, right)
ax.axvline(0, color="grey", linewidth=1, zorder=0)

footer("Anmerkung: Δ in Prozentpunkten (pp). Positiv = Wachstum, Negativ = Rückgang. Quelle: Statista.")
save_fig(FIG / "03_delta_ranking.png")


In [35]:
# 03_reporting — Cell 7: Dumbbell 2021→2024
import numpy as np

if {2021, 2024}.issubset(set(stat_pivot.columns)):
    order = delta.index  # gleiche Reihenfolge wie im Δ-Ranking
    y = np.arange(len(order))

    fig, ax = plt.subplots(figsize=(12, 6), layout="constrained")  # <- INSIDE dem if-Block!
    x1 = stat_pivot.loc[order, 2021].values
    x3 = stat_pivot.loc[order, 2024].values

    for yi, a, b in zip(y, x1, x3):
        ax.plot([a, b], [yi, yi], marker="o")

    ax.set_yticks(y)
    ax.set_yticklabels(order)
    ax.set_title("Entwicklung 2021→2024 je Kategorie (Dumbbell)")
    ax.set_xlabel("Anteil in %")
    percent_axis(ax, axis="x", decimals=0, limit=(0, 100), pad_pct=5)  # 5 pp Luft rechts

    footer("Quelle: Statista (2021, 2024). Einheit: Prozent. Hinweis: 2022 im Diagramm nicht dargestellt; siehe Gruppenbalken.")
    save_fig(FIG / "03_dumbbell_2021_2024.png")
else:
    print("Dumbbell-Plot übersprungen: Jahre 2021/2024 nicht vollständig vorhanden.")


In [51]:
# 03_reporting — Cell 8: Bedarfsgüter vs. Luxusgüter (absolute Entwicklung 2021/2022/2024)

NEEDS = {"Lebensmittel / Getränke", "Medikamente / Drogerieartikel"}
LUXURY = {
    "Kleidung / Schuhe",
    "Elektronik (z. B. Smartphones, Haushaltsgeräte)",
    "Bücher / Medien / Software",
    "Hobby- & Freizeitartikel",
    "Möbel / Wohnaccessoires",
}

unknown = set(KANON) - NEEDS - LUXURY
if unknown:
    print("Hinweis: nicht zugeordnet (werden ignoriert):", unknown)

def group_share(df_wide: pd.DataFrame, group: set[str]) -> pd.Series:
    common = [c for c in df_wide.index if c in group]
    # Summe über die in YEARS verfügbaren Spalten (Werte in %)
    return df_wide.loc[common, YEARS].sum()

needs_share = group_share(stat_pivot, NEEDS)
lux_share   = group_share(stat_pivot, LUXURY)

need_lux_df = pd.DataFrame(
    {"Bedarfsgüter": needs_share, "Luxusgüter": lux_share},
    index=YEARS
).sort_index()
display(need_lux_df)

# ---------- Plot: absolute Entwicklung als Zeitreihe ----------
fig, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)

# Linien (alle drei Jahre) mit Markern
ax.plot(need_lux_df.index, need_lux_df["Bedarfsgüter"], marker="o", linewidth=2, label="Bedarfsgüter", zorder=3)
ax.plot(need_lux_df.index, need_lux_df["Luxusgüter"],   marker="o", linewidth=2, label="Luxusgüter",   zorder=3)

# Prozentlabels an die Punkte (mit Headroom, ohne Überlappung)
ymax = float(np.nanmax(need_lux_df.values))
for x, y in zip(need_lux_df.index, need_lux_df["Bedarfsgüter"].values):
    ax.text(x, y + max(1, ymax*0.02), f"{y:.1f}%", ha="center", va="bottom", fontsize=9, clip_on=False)
for x, y in zip(need_lux_df.index, need_lux_df["Luxusgüter"].values):
    ax.text(x, y + max(1, ymax*0.02), f"{y:.1f}%", ha="center", va="bottom", fontsize=9, clip_on=False)

# Achsen, Raster, Legende
ax.set_title("Anteile Bedarfsgüter vs. Luxusgüter – absolute Entwicklung (2021 / 2022 / 2024)")
ax.set_xlabel("Jahr")
ax.set_ylabel("Anteil in %")
percent_axis(ax, axis="y", decimals=0, limit=(0, max(100, ymax*1.15)))  # 0–100 oder mit Headroom
ax.set_xticks(need_lux_df.index)
ax.grid(True, linestyle=":", alpha=0.3)
ax.legend(frameon=False, ncols=2, loc="lower center", bbox_to_anchor=(0.5, 1.02))

# Fußnote außerhalb der Achse
fig.text(0.01, -0.04,
         "Summen über Kategorien (Mehrfachnennungen möglich) – keine 100%-Verteilung pro Jahr. Quelle: Statista (2021, 2022, 2024).",
         ha="left", va="top", fontsize=9, color="#555")

save_fig(FIG / "03_needs_vs_luxury_lines_absolute.png")

# Δ-Tabelle (wie bisher)
shift_df = pd.DataFrame({
    "Δ Bedarf (2021→2024)": [needs_share.get(2024, float("nan")) - needs_share.get(2021, float("nan"))],
    "Δ Luxus (2021→2024)":  [lux_share.get(2024, float("nan"))  - lux_share.get(2021, float("nan"))]
})
display(shift_df.style.format("{:+.1f} pp"))



,Bedarfsgüter,Luxusgüter
2021,111.0,216.0
2022,120.0,418.0
2024,67.0,241.0


,Δ Bedarf (2021→2024),Δ Luxus (2021→2024)
0,-44.0 pp,+25.0 pp


In [37]:

# 03_reporting — Cell 9: Text-Antworten zu den Leitfragen
def top_changes(delta_df: pd.DataFrame, k=3):
    col = delta_df.columns[-1]
    inc = delta_df[col].nlargest(k)
    dec = delta_df[col].nsmallest(k)
    return col, inc, dec

col, inc, dec = top_changes(delta, k=3)
def bullets(series) -> list[str]:
    return [f"- {idx}: {val:+.1f} pp ({col})" for idx, val in series.items()]

answers = []
answers.append("Frage: **Welche Produktkategorien wurden mehr oder weniger gekauft?**")
answers.append("Mehr gekauft (Top-3):\n" + "\n".join(bullets(inc)))
answers.append("Weniger gekauft (Top-3):\n" + "\n".join(bullets(dec)))

answers.append("Frage: **In welchen Warengruppen gab es Wachstum oder Rückgang?**")
wachs = delta[delta[col] > 0].index.tolist()
rueck = delta[delta[col] < 0].index.tolist()
answers.append("Wachstum: " + (", ".join(wachs) if wachs else "–"))
answers.append("Rückgang: " + (", ".join(rueck) if rueck else "–"))

answers.append("Frage: **Gibt es Verschiebungen zwischen Bedarfsgütern und Luxusgütern?**")
if set([2021,2024]).issubset(need_lux_df.index):
    answers.append(f"- Bedarfsgüter gesamt: 2021 = {need_lux_df.loc[2021,'Bedarfsgüter']:.1f} %, 2024 = {need_lux_df.loc[2024,'Bedarfsgüter']:.1f} % → Δ = {need_lux_df.loc[2024,'Bedarfsgüter']-need_lux_df.loc[2021,'Bedarfsgüter']:+.1f} pp")
    answers.append(f"- Luxusgüter gesamt:  2021 = {need_lux_df.loc[2021,'Luxusgüter']:.1f} %, 2024 = {need_lux_df.loc[2024,'Luxusgüter']:.1f} % → Δ = {need_lux_df.loc[2024,'Luxusgüter']-need_lux_df.loc[2021,'Luxusgüter']:+.1f} pp")
else:
    answers.append("- Daten für 2021/2024 unvollständig; bitte prüfen.")
answers.append("Interpretation: **Prozentpunkte (pp)** ≠ Prozent. Positive Δ = Anteil stieg; negative Δ = Anteil sank.")

print("\n\n".join(answers))


Frage: **Welche Produktkategorien wurden mehr oder weniger gekauft?**

Mehr gekauft (Top-3):
- Kleidung / Schuhe: +49.0 pp (Δ 2021→2024)
- Bücher / Medien / Software: +25.0 pp (Δ 2021→2024)
- Elektronik (z. B. Smartphones, Haushaltsgeräte): +0.0 pp (Δ 2021→2024)

Weniger gekauft (Top-3):
- Medikamente / Drogerieartikel: -27.0 pp (Δ 2021→2024)
- Hobby- & Freizeitartikel: -26.0 pp (Δ 2021→2024)
- Möbel / Wohnaccessoires: -23.0 pp (Δ 2021→2024)

Frage: **In welchen Warengruppen gab es Wachstum oder Rückgang?**

Wachstum: Kleidung / Schuhe, Bücher / Medien / Software

Rückgang: Lebensmittel / Getränke, Möbel / Wohnaccessoires, Hobby- & Freizeitartikel, Medikamente / Drogerieartikel

Frage: **Gibt es Verschiebungen zwischen Bedarfsgütern und Luxusgütern?**

- Bedarfsgüter gesamt: 2021 = 111.0 %, 2024 = 67.0 % → Δ = -44.0 pp

- Luxusgüter gesamt:  2021 = 216.0 %, 2024 = 241.0 % → Δ = +25.0 pp

Interpretation: **Prozentpunkte (pp)** ≠ Prozent. Positive Δ = Anteil stieg; negative Δ = Anteil sa

In [38]:

# 03_reporting — Cell 10: Kurzbefund exportieren
REPORT_TXT = OUT / "reporting_kurzbefund.txt"
with open(REPORT_TXT, "w", encoding="utf-8") as f:
    f.write("\n\n".join(answers))
print("Kurzbefund exportiert →", REPORT_TXT)
print("Fertige Abbildungen →", FIG)


Kurzbefund exportiert → D:\Q3_2025\data-analytics\project\data\processed\reporting_kurzbefund.txt
Fertige Abbildungen → D:\Q3_2025\data-analytics\project\reports\figures
